In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark K-means example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/18 17:19:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/18 17:19:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
from pyspark.ml.clustering import GaussianMixture


In [3]:
df = spark.read.format('csv').\
                       options(header='true', \
                       inferschema='true').\
            load("data/iris.csv",header=True);

In [4]:
df.show(5,True)
df.printSchema()

+------------+-----------+------------+-----------+-------+
|sepal_length|sepal_width|petal_length|petal_width|species|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2| setosa|
|         4.9|        3.0|         1.4|        0.2| setosa|
|         4.7|        3.2|         1.3|        0.2| setosa|
|         4.6|        3.1|         1.5|        0.2| setosa|
|         5.0|        3.6|         1.4|        0.2| setosa|
+------------+-----------+------------+-----------+-------+
only showing top 5 rows

root
 |-- sepal_length: double (nullable = true)
 |-- sepal_width: double (nullable = true)
 |-- petal_length: double (nullable = true)
 |-- petal_width: double (nullable = true)
 |-- species: string (nullable = true)



In [5]:
df.describe().show()

+-------+------------------+-------------------+------------------+------------------+---------+
|summary|      sepal_length|        sepal_width|      petal_length|       petal_width|  species|
+-------+------------------+-------------------+------------------+------------------+---------+
|  count|               150|                150|               150|               150|      150|
|   mean| 5.843333333333335| 3.0540000000000007|3.7586666666666693|1.1986666666666672|     null|
| stddev|0.8280661279778637|0.43359431136217375| 1.764420419952262|0.7631607417008414|     null|
|    min|               4.3|                2.0|               1.0|               0.1|   setosa|
|    max|               7.9|                4.4|               6.9|               2.5|virginica|
+-------+------------------+-------------------+------------------+------------------+---------+



### We do not care the target label, as this is clustering, we are supposed to generate label

In [7]:
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors

transformed=df.rdd.map(lambda r: [Vectors.dense(r[:-1]),r[-1]]).toDF(['features','label']).select("features")
transformed.show(5)

+-----------------+
|         features|
+-----------------+
|[5.1,3.5,1.4,0.2]|
|[4.9,3.0,1.4,0.2]|
|[4.7,3.2,1.3,0.2]|
|[4.6,3.1,1.5,0.2]|
|[5.0,3.6,1.4,0.2]|
+-----------------+
only showing top 5 rows



Deal With Categorical Variables

In [8]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

featureIndexer = VectorIndexer(inputCol="features", \
                               outputCol="indexedFeatures",\
                               maxCategories=4).fit(transformed)

data = featureIndexer.transform(transformed)
data.show(5,True)

+-----------------+-----------------+
|         features|  indexedFeatures|
+-----------------+-----------------+
|[5.1,3.5,1.4,0.2]|[5.1,3.5,1.4,0.2]|
|[4.9,3.0,1.4,0.2]|[4.9,3.0,1.4,0.2]|
|[4.7,3.2,1.3,0.2]|[4.7,3.2,1.3,0.2]|
|[4.6,3.1,1.5,0.2]|[4.6,3.1,1.5,0.2]|
|[5.0,3.6,1.4,0.2]|[5.0,3.6,1.4,0.2]|
+-----------------+-----------------+
only showing top 5 rows



In [9]:
data.createOrReplaceTempView("data")
data=spark.sql("select indexedFeatures as features from data")
data.show(5)


+-----------------+
|         features|
+-----------------+
|[5.1,3.5,1.4,0.2]|
|[4.9,3.0,1.4,0.2]|
|[4.7,3.2,1.3,0.2]|
|[4.6,3.1,1.5,0.2]|
|[5.0,3.6,1.4,0.2]|
+-----------------+
only showing top 5 rows



In [10]:
# Trains Gaussian Mixture Model
gmm = GaussianMixture(k=3)

### clustering algorithm is for unsupervised learning, therefore, no training and testing splits necessary

In [11]:
# Chain indexer and tree in a Pipeline
pipeline = Pipeline(stages=[gmm])
model = pipeline.fit(data)


23/05/18 17:19:36 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
23/05/18 17:19:36 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK
23/05/18 17:19:36 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/05/18 17:19:36 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
23/05/18 17:19:36 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/05/18 17:19:36 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


In [12]:
cluster = model.transform(data)

In [13]:
cluster.show(truncate=False)

+-----------------+-----------------------------------------------------------------+----------+
|features         |probability                                                      |prediction|
+-----------------+-----------------------------------------------------------------+----------+
|[5.1,3.5,1.4,0.2]|[4.734799396261589E-17,0.9999999997849096,2.1509033078181288E-10]|1         |
|[4.9,3.0,1.4,0.2]|[8.242255300723322E-15,0.9999997487664355,2.5123355615813866E-7] |1         |
|[4.7,3.2,1.3,0.2]|[3.704702543514495E-16,0.9999999338662796,6.613371996012292E-8]  |1         |
|[4.6,3.1,1.5,0.2]|[1.7264949151793987E-13,0.9999998802939727,1.197058546285927E-7] |1         |
|[5.0,3.6,1.4,0.2]|[5.785073474861345E-17,0.9999999999088787,9.112123028276783E-11] |1         |
|[5.4,3.9,1.7,0.4]|[2.437845167521415E-16,0.9999999997220376,2.779621827766594E-10] |1         |
|[4.6,3.4,1.4,0.3]|[5.079352293083531E-16,0.9999998192072068,1.8079279267312214E-7] |1         |
|[5.0,3.4,1.5,0.2]|[1.21646667

In [14]:
gaussianM=model.stages[0]
for i in range(gaussianM.getK()):
    print(f"{gaussianM.weights[i]}, {gaussianM.gaussians[i].mean}, {gaussianM.gaussians[i].cov}")

0.3679542368270777, [6.222739873722937,2.9544088011137513,5.078348688349975,1.858811639230237], DenseMatrix([[0.27354254, 0.06839109, 0.22347098, 0.12722684],
             [0.06839109, 0.06450981, 0.06277417, 0.05153339],
             [0.22347098, 0.06277417, 0.31957783, 0.18490413],
             [0.12722684, 0.05153339, 0.18490413, 0.1583885 ]])
0.3331255382005878, [5.006313815873003,3.4186934435434684,1.4641011574974798,0.243964220238869], DenseMatrix([[0.12168099, 0.09800223, 0.01577433, 0.01036   ],
             [0.09800223, 0.14158903, 0.01134136, 0.01125382],
             [0.01577433, 0.01134136, 0.0295056 , 0.005593  ],
             [0.01036   , 0.01125382, 0.005593  , 0.01126874]])
0.2989202249723345, [6.309104203253557,2.770166090515914,4.691342865011557,1.450013415518377], DenseMatrix([[0.63138072, 0.19500516, 0.74876637, 0.23363146],
             [0.19500516, 0.14656056, 0.20009054, 0.07224747],
             [0.74876637, 0.20009054, 1.03789818, 0.32604766],
             [0.2

https://stats.stackexchange.com/questions/192168/is-the-covariance-matrix-the-equivalent-of-standard-deviation-for-a-2d-matrix